<a href="https://colab.research.google.com/github/sagarikajadon13/Grammar-Error-Detection/blob/main/2_LGBMClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize

import spacy
import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import f1_score, accuracy_score

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
import lightgbm as lgb

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
train= pd.read_csv('/content/drive/MyDrive/SHL NLP task/train_processed.csv')
val= pd.read_csv('/content/drive/MyDrive/SHL NLP task/val_data.csv')
test= pd.read_csv('/content/drive/MyDrive/SHL NLP task/test_data.csv')
test_inputs= test['input']

In [ ]:
X_train= train['input']
y_train= train['labels']

X_val= val['input']
y_val= val['labels']

test= test['input']

In [ ]:
vectorizer= TfidfVectorizer(lowercase= False, tokenizer= sent_tokenize)
vectorizer.fit(X_train)
X_train= vectorizer.transform(X_train)
X_val= vectorizer.transform(X_val)
test= vectorizer.transform(test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
model= lgb.LGBMClassifier(objective= 'binary',
                          max_depth= 3,
                          learning_rate= 0.1,
                          n_estimators= 1000)

In [ ]:
model.fit(X_train, y_train,
          eval_set= [(X_val, y_val)],
          eval_metric= 'logloss',
          early_stopping_rounds= 100,
          verbose= 100)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.693566	valid_0's binary_logloss: 0.693566
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.693133	valid_0's binary_logloss: 0.693133


LGBMClassifier(max_depth=3, n_estimators=1000, objective='binary')

In [ ]:
y_pred= model.predict(X_val)
f1_score(y_val, y_pred)

0.6641061923051007

In [ ]:
test_preds_lgbm= model.predict(test)

In [ ]:
sub= pd.DataFrame({'input': test_inputs,
                   'labels': test_preds_lgbm})

In [ ]:
sub.to_csv('submission_lgbm.csv', index= False)